# Building Tools

LLM agents are one of the most powerful and fascinating technologies to come out of the huge explosion of LLMs since the introduction of ChatGPT.

Using agents we are able to give LLMs access to tools. These tools give us an essentially infinite number of possibilities as they allow us to search the web, do maths, run code, and much more.

There are many prebuilt tools available from LangChain, but in many real world scenarios we need to build custom tools to fit the requirements of our use-cases.

In this notebook, we will explore how we can build custom tools for agents in LangChain. Starting with simple math functions to give us an idea of how it works, before moving on to using other ML models to give us even more abilities like describing images.

In [1]:
# !pip install transformers

## Building Tools

Let's begin taking a look at how we can build custom tools. We'll start with some simple examples so we can understand the flow of the tool building process.

### Simple Calculator Tool

We will begin with a very simple circumference calculating tool that we will define like so:

In [2]:
from langchain.tools import BaseTool
from math import pi
from typing import Union


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius)*2.0*pi
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")

When defining a tool like this using the `BaseTool` template we must define name and description attributes, alongside `_run` and `_arun` methods. When a tool us used the `_run` method is called by default. The `_arun` method is called when the tool is to be used `asyncronously`. We do not cover that in this walkthrough so for now we create it with a `NotImplementedError`.

We need to initialize the agent `LLM` and `conversational memory`. For the LLM we will use `gpt-3.5-turbo` from OpenAI, to use this you need an OpenAI API key.

In [3]:
import inspect

from getpass import getpass
from langchain import OpenAI,PromptTemplate 
import tiktoken
from langchain.chat_models import AzureChatOpenAI
import os

In [4]:
import os
import openai

os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01" #"2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = openai.api_base =  "https://idocopenaigpt.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "95776649ac7a4b048c834003fd315264"#os.getenv("AZUREOPENAI_API_KEY")

In [5]:
llm = AzureChatOpenAI(
    deployment_name="chatllm16k",
    openai_api_version="2023-03-15-preview",
    model_name="gpt-3.5-turbo",
    temperature=0,
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureChatOpenAI instead.
  warn_deprecated(
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:165: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://idocopenaigpt.openai.azure.com/ to https://idocopenaigpt.openai.azure.com/openai.
  warnings.warn(
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:172: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_na

In [6]:

from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)


Now we initialize the agent itself. To use the circumference tool we defined above we must add it to a list that we then pass to the `tools` parameter within `initialize_agent`:

In [7]:
from langchain.agents import initialize_agent

tools = [CircumferenceTool()]

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


Now let's try and ask it to calculate the circumference of a circle:

In [8]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
Observation: 49.071677249072565
Thought:{
    "action": "Final Answer",
    "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}

In [9]:

(7.81 * 2) * pi
     

49.071677249072565

The answer is pretty close but not quite right. We can see in the outputs above that the model decided not to use the circumference calculator tool, that is because it is (falsely) confident in it's ability to answer the question.

To fix this we need to tell the model in the prompt that it cannot do this. We do this like so:

In [10]:
# existing prompt
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

Let's modify the existing system message to emphasize that the LLM is terrible at maths (fourth line):

In [17]:
sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Assistant understands string operationa and can convert or read numerical string into numbers and perform actions.

Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""
     

And update our prompt.

In [18]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [19]:
#Now we try again

In [22]:
agent("can you calculate the circumference of a circle that has a radius of 7.81 mm")



> Entering new AgentExecutor chain...
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
Observation: 49.071677249072565
Thought:{
    "action": "Final Answer",
    "action_input": "49.071677249072565"
}

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81 mm',
 'chat_history': [HumanMessage(content='can you calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='can you calculate the circumference of a circle that has a radius of 7 mm'),
  AIMessage(content='43.982297150257104')],
 'output': '49.071677249072565'}

### Tools with Multiple Parameters

Tools can also handle multiple parameters, let's try and build a Pythagoras tool. The tool helps us calculate the hypotenuse of a triangle given a combination of side lengths and/or angles.



In [23]:
from typing import Optional
from math import sqrt, cos, sin

desc = (
    "use this tool when you need to calculate the length of an hypotenuse "
    "given one or two sides of a triangle and/or an angle (in degrees). "
    "To use the tool you must provide at least two of the following parameters "
    "['adjacent_side', 'opposite_side', 'angle']."
)

class PythagorasTool(BaseTool):
    name = "Hypotenuse calculator"
    description = desc

    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        # check for the values we have been given
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

tools = [PythagorasTool()]

In [24]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt
# update the agent tools
agent.tools = tools

In [25]:
agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...
{
    "action": "Hypotenuse calculator",
    "action_input": {
        "adjacent_side": 51,
        "opposite_side": 34
    }
}
Observation: 61.29437168288782
Thought:{
    "action": "Final Answer",
    "action_input": "The length of the hypotenuse is approximately 61.29 cm."
}

> Finished chain.


{'input': 'If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?',
 'chat_history': [HumanMessage(content='can you calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='can you calculate the circumference of a circle that has a radius of 7 mm'),
  AIMessage(content='43.982297150257104'),
  HumanMessage(content='can you calculate the circumference of a circle that has a radius of 7.81 mm'),
  AIMessage(content='49.071677249072565')],
 'output': 'The length of the hypotenuse is approximately 61.29 cm.'}